In [ ]:
import config
from phemex import *
import json
import pandas as pd
from datetime import datetime
import nest_asyncio
import time

nest_asyncio.apply()
import asyncio
import websockets
clients = {}
# Create an instance of the Phemex exchange
exchange = initExchange()
symbol = config.TRADE_SYMBOL
trades = pd.DataFrame(columns=['timestamp', 'side', 'priceEp', 'volume'])
tradesPrices = []
ohlcvData = pd.DataFrame(
    columns=['open', 'high', 'low', 'close', 'volume', 'timestamp'])
klineData = pd.DataFrame(
    columns=['timestamp', 'interval', 'lastCloseEp', 'openEp', 'highEp', 'lowEp', 'closeEp', 'volume', 'turnoverEv'])
tick_size = 0.5
quantity = 100
stop_distance = 10

# Set up exchange client
exchange = initExchange()

In [ ]:
async def strategy(lastMbtcTick,lastClose):
    position = getPositionInfo()
    # print(f'position: {position}')
    # Check if .MBTC ticker is above last price
    if lastMbtcTick > lastClose:
        # Check if there is a short position
        if position['side'] == 'Sell':                
            # Remove stop loss
            exchange.cancel_all_orders({'symbol': config.TRADE_SYMBOL, 'side': 'Buy'})
            print('Removed stop loss')        
        # Check if there is a long position
        elif position['side'] == 'Buy':
            # Set stop loss slightly above liquidation price
            stop_price = position['liquidationPrice'] + stop_distance
            exchange.create_post_only_order({
                'symbol': config.TRADE_SYMBOL,
                'side': 'Sell',
                'orderQty': quantity,
                'price': lastClose - tick_size,
                'stopPx': stop_price,
                'execInst': 'LastPrice',
            })
            print('Set stop loss at', stop_price)        
        # If there is no position, create a long position
        elif position['amount'] == 0:
            exchange.create_post_only_order({
                'symbol': config.TRADE_SYMBOL,
                'side': 'Buy',
                'orderQty': quantity,
                'execInst': 'LastPrice',
            })
            print('Created long position')   
 
    # Check if .MBTC ticker is below last price
    elif lastMbtcTick < lastClose:
        # Check if there is a long position
        if position['side'] == 'Buy':     
            # Remove stop loss
            exchange.cancel_all_orders({'symbol': config.TRADE_SYMBOL, 'side': 'Sell'})
            print('Removed stop loss')                 
        # Check if there is a short position
        if position['side'] == 'Sell':    
            # Set stop loss slightly below liquidation price
            stop_price = position['liquidationPrice'] - stop_distance
            exchange.create_post_only_order({
                'symbol': config.TRADE_SYMBOL,
                'side': 'Buy',
                'orderQty': quantity,
                'price': lastClose + tick_size,
                'stopPx': stop_price,
                'execInst': 'LastPrice',
            })
            print('Set stop loss at', stop_price)      
        # If there is no position, create a short position
        elif position['amount'] == 0:
            exchange.create_post_only_order({
                'symbol': config.TRADE_SYMBOL,
                'side': 'Sell',
                'orderQty': quantity,
                'execInst': 'LastPrice',
            })
            print('Created short position')
 

In [ ]:
klines = []
ticks = []
grouped_data = {}
tick_size = 0.5
quantity = 100
stop_distance = 10
async def send_message(message):

    global ohlcvData, tradesPrices, klineData, klines
    if message != None:
        # print(message)
        if 'kline' in message:
            # print(json.loads(message)['type'])
            if json.loads(message)['type'] == 'snapshot':
                print(json.loads(message)['kline'])
                
            for kline in json.loads(message)['kline']:
                klines.append(kline)    
            
            for item in klines:
                
                timestamp = datetime.fromtimestamp(datetime.now().timestamp()).strftime("%Y-%m-%d %H:%M:%S")
                if timestamp not in grouped_data:
                    grouped_data[timestamp] = []
                
                grouped_data[timestamp].append(item)
        print('lengte kline: ',len(klines))
        if 'tick' in message:
            ticks.append(json.loads(message)['tick'])  
            if len(klines) > 2:    
                lastMbtcTick = ticks[-1]['last']
                secondLastMbtcTick = ticks[-2]['last']
                lastClose = klines[-1]
                secondLastClose = klines[-2]
                await strategy(lastMbtcTick,lastClose)
                # await mbtcStrategy(lastMbtcTick,secondLastMbtcTick,lastClose,secondLastClose)
        # if len(klines) > 100:
        #     await sarStrategy(klines)       
        for clientId in clients:
            client = clients[clientId]
            sendMessage = json.dumps({'klines': [grouped_data],'ticks':ticks})
            await client['websocket'].send(sendMessage)

In [ ]:
# connect to Phemes websocket server
async def connectPhemexWS():
    global interval
    async for websocket in websockets.connect('wss://vapi.phemex.com/ws'):
        try:
            print('Phemex Websocket Server is connected!')
            subscribe_ticker_msg = json.dumps({
                "id": 0,
                "method": "tick.subscribe",
                "params": [config.MBTC_SYMBOL]
            })
            await websocket.send(subscribe_ticker_msg)

            subscribe_kline_msg = json.dumps({
                "id": 0,
                "method": "kline.subscribe",
                "params": [config.TRADE_SYMBOL_BTCUSD,60]
            })
            await websocket.send(subscribe_kline_msg)

            while True:
                message = await websocket.recv()            
                # if 'book' in message:
                #     print(message)
                await send_message(message)
        except websockets.ConnectionClosed:
            continue

In [ ]:
async def register(websocket):
    # Assign a unique ID to the client
    client_id = len(clients) + 1
    clients[client_id] = {"websocket": websocket, "info": {}}
    # await websocket.send(f"You are client {client_id}")
    print(f"Client {client_id} connected")
    return client_id

async def unregister(client_id):
    # Remove the client from the clients array
    del clients[client_id]
    print(f"Client {client_id} disconnected")

async def handle_message(client_id, message):
    # Handle incoming messages from the client
    print(f"Received message from client {client_id}: {message}")
    # Store extra client information in clients array
    clients[client_id]["info"] = {"extra_info": "example"}

async def handle_client(websocket, path):
    client_id = await register(websocket)
    try:
        async for message in websocket:
            await handle_message(client_id, message)
    except websockets.exceptions.ConnectionClosed:
        pass
    finally:
        # Unregister the client when they disconnect
        await unregister(client_id)

In [ ]:
# start python localhost websocket server
async def start_server():
    try:
        async with websockets.serve(handle_client, host="localhost", port=8765):
            print("Server started")
            await connectPhemexWS()
            await asyncio.Future()  # Keep the server running
    except Exception as e:
        print('Error {}'.format(e))

In [ ]:
if __name__ == '__main__':
    asyncio.run(start_server())